In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import torch

import warnings; warnings.simplefilter('ignore')

from brainpedia.brainpedia import BrainPedia
from brainpedia.fmri_processing import invert_preprocessor_scaling
from models.ICW_FMRI_GAN import Critic, Generator
from nilearn import plotting
from torch.autograd import Variable
from tqdm import tqdm

Fix random seed for reproducibility.

In [2]:
np.random.seed(1)

CUDA = torch.cuda.is_available()
if CUDA:
    torch.cuda.manual_seed(1)
    print("CUDA is available...")
else:
    torch.manual_seed(1)
    print("CUDA is NOT available...")

CUDA is available...


# Data

Download the full [Neurovault](https://neurovault.org) Collection 1952 [BrainPedia](https://neurovault.org/collections/1952/) dataset.

In [3]:
BrainPedia.downloadIfNecessary()

Reading local neurovault data.
Already fetched 1 image
Already fetched 2 images
Already fetched 3 images
Already fetched 4 images
Already fetched 5 images
Already fetched 6 images
Already fetched 7 images
Already fetched 8 images
Already fetched 9 images
Already fetched 10 images
Already fetched 11 images
Already fetched 12 images
Already fetched 13 images
Already fetched 14 images
Already fetched 15 images
Already fetched 16 images
Already fetched 17 images
Already fetched 18 images
Already fetched 19 images
Already fetched 20 images
Already fetched 21 images
Already fetched 22 images
Already fetched 23 images
Already fetched 24 images
Already fetched 25 images
Already fetched 26 images
Already fetched 27 images
Already fetched 28 images
Already fetched 29 images
Already fetched 30 images
Already fetched 31 images
Already fetched 32 images
Already fetched 33 images
Already fetched 34 images
Already fetched 35 images
Already fetched 36 images
Already fetched 37 images
Already fetched 3

Already fetched 559 images
Already fetched 560 images
Already fetched 561 images
Already fetched 562 images
Already fetched 563 images
Already fetched 564 images
Already fetched 565 images
Already fetched 566 images
Already fetched 567 images
Already fetched 568 images
Already fetched 569 images
Already fetched 570 images
Already fetched 571 images
Already fetched 572 images
Already fetched 573 images
Already fetched 574 images
Already fetched 575 images
Already fetched 576 images
Already fetched 577 images
Already fetched 578 images
Already fetched 579 images
Already fetched 580 images
Already fetched 581 images
Already fetched 582 images
Already fetched 583 images
Already fetched 584 images
Already fetched 585 images
Already fetched 586 images
Already fetched 587 images
Already fetched 588 images
Already fetched 589 images
Already fetched 590 images
Already fetched 591 images
Already fetched 592 images
Already fetched 593 images
Already fetched 594 images
Already fetched 595 images
A

Already fetched 871 images
Already fetched 872 images
Already fetched 873 images
Already fetched 874 images
Already fetched 875 images
Already fetched 876 images
Already fetched 877 images
Already fetched 878 images
Already fetched 879 images
Already fetched 880 images
Already fetched 881 images
Already fetched 882 images
Already fetched 883 images
Already fetched 884 images
Already fetched 885 images
Already fetched 886 images
Already fetched 887 images
Already fetched 888 images
Already fetched 889 images
Already fetched 890 images
Already fetched 891 images
Already fetched 892 images
Already fetched 893 images
Already fetched 894 images
Already fetched 895 images
Already fetched 896 images
Already fetched 897 images
Already fetched 898 images
Already fetched 899 images
Already fetched 900 images
Already fetched 901 images
Already fetched 902 images
Already fetched 903 images
Already fetched 904 images
Already fetched 905 images
Already fetched 906 images
Already fetched 907 images
A

Already fetched 1172 images
Already fetched 1173 images
Already fetched 1174 images
Already fetched 1175 images
Already fetched 1176 images
Already fetched 1177 images
Already fetched 1178 images
Already fetched 1179 images
Already fetched 1180 images
Already fetched 1181 images
Already fetched 1182 images
Already fetched 1183 images
Already fetched 1184 images
Already fetched 1185 images
Already fetched 1186 images
Already fetched 1187 images
Already fetched 1188 images
Already fetched 1189 images
Already fetched 1190 images
Already fetched 1191 images
Already fetched 1192 images
Already fetched 1193 images
Already fetched 1194 images
Already fetched 1195 images
Already fetched 1196 images
Already fetched 1197 images
Already fetched 1198 images
Already fetched 1199 images
Already fetched 1200 images
Already fetched 1201 images
Already fetched 1202 images
Already fetched 1203 images
Already fetched 1204 images
Already fetched 1205 images
Already fetched 1206 images
Already fetched 1207

Already fetched 1483 images
Already fetched 1484 images
Already fetched 1485 images
Already fetched 1486 images
Already fetched 1487 images
Already fetched 1488 images
Already fetched 1489 images
Already fetched 1490 images
Already fetched 1491 images
Already fetched 1492 images
Already fetched 1493 images
Already fetched 1494 images
Already fetched 1495 images
Already fetched 1496 images
Already fetched 1497 images
Already fetched 1498 images
Already fetched 1499 images
Already fetched 1500 images
Already fetched 1501 images
Already fetched 1502 images
Already fetched 1503 images
Already fetched 1504 images
Already fetched 1505 images
Already fetched 1506 images
Already fetched 1507 images
Already fetched 1508 images
Already fetched 1509 images
Already fetched 1510 images
Already fetched 1511 images
Already fetched 1512 images
Already fetched 1513 images
Already fetched 1514 images
Already fetched 1515 images
Already fetched 1516 images
Already fetched 1517 images
Already fetched 1518

...

Already fetched 4837 images
Already fetched 4838 images
Already fetched 4839 images
Already fetched 4840 images
Already fetched 4841 images
Already fetched 4842 images
Already fetched 4843 images
Already fetched 4844 images
Already fetched 4845 images
Already fetched 4846 images
Already fetched 4847 images
Already fetched 4848 images
Already fetched 4849 images
Already fetched 4850 images
Already fetched 4851 images
Already fetched 4852 images
Already fetched 4853 images
Already fetched 4854 images
Already fetched 4855 images
Already fetched 4856 images
Already fetched 4857 images
Already fetched 4858 images
Already fetched 4859 images
Already fetched 4860 images
Already fetched 4861 images
Already fetched 4862 images
Already fetched 4863 images
Already fetched 4864 images
Already fetched 4865 images
Already fetched 4866 images
Already fetched 4867 images
Already fetched 4868 images
Already fetched 4869 images
Already fetched 4870 images
Already fetched 4871 images
Already fetched 4872

Already fetched 5169 images
Already fetched 5170 images
Already fetched 5171 images
Already fetched 5172 images
Already fetched 5173 images
Already fetched 5174 images
Already fetched 5175 images
Already fetched 5176 images
Already fetched 5177 images
Already fetched 5178 images
Already fetched 5179 images
Already fetched 5180 images
Already fetched 5181 images
Already fetched 5182 images
Already fetched 5183 images
Already fetched 5184 images
Already fetched 5185 images
Already fetched 5186 images
Already fetched 5187 images
Already fetched 5188 images
Already fetched 5189 images
Already fetched 5190 images
Already fetched 5191 images
Already fetched 5192 images
Already fetched 5193 images
Already fetched 5194 images
Already fetched 5195 images
Already fetched 5196 images
Already fetched 5197 images
Already fetched 5198 images
Already fetched 5199 images
Already fetched 5200 images
Already fetched 5201 images
Already fetched 5202 images
Already fetched 5203 images
Already fetched 5204

Already fetched 5679 images
Already fetched 5680 images
Already fetched 5681 images
Already fetched 5682 images
Already fetched 5683 images
Already fetched 5684 images
Already fetched 5685 images
Already fetched 5686 images
Already fetched 5687 images
Already fetched 5688 images
Already fetched 5689 images
Already fetched 5690 images
Already fetched 5691 images
Already fetched 5692 images
Already fetched 5693 images
Already fetched 5694 images
Already fetched 5695 images
Already fetched 5696 images
Already fetched 5697 images
Already fetched 5698 images
Already fetched 5699 images
Already fetched 5700 images
Already fetched 5701 images
Already fetched 5702 images
Already fetched 5703 images
Already fetched 5704 images
Already fetched 5705 images
Already fetched 5706 images
Already fetched 5707 images
Already fetched 5708 images
Already fetched 5709 images
Already fetched 5710 images
Already fetched 5711 images
Already fetched 5712 images
Already fetched 5713 images
Already fetched 5714

Already fetched 5976 images
Already fetched 5977 images
Already fetched 5978 images
Already fetched 5979 images
Already fetched 5980 images
Already fetched 5981 images
Already fetched 5982 images
Already fetched 5983 images
Already fetched 5984 images
Already fetched 5985 images
Already fetched 5986 images
Already fetched 5987 images
Already fetched 5988 images
Already fetched 5989 images
Already fetched 5990 images
Already fetched 5991 images
Already fetched 5992 images
Already fetched 5993 images
Already fetched 5994 images
Already fetched 5995 images
Already fetched 5996 images
Already fetched 5997 images
Already fetched 5998 images
Already fetched 5999 images
Already fetched 6000 images
Already fetched 6001 images
Already fetched 6002 images
Already fetched 6003 images
Already fetched 6004 images
Already fetched 6005 images
Already fetched 6006 images
Already fetched 6007 images
Already fetched 6008 images
Already fetched 6009 images
Already fetched 6010 images
Already fetched 6011

Already fetched 6295 images
Already fetched 6296 images
Already fetched 6297 images
Already fetched 6298 images
Already fetched 6299 images
Already fetched 6300 images
Already fetched 6301 images
Already fetched 6302 images
Already fetched 6303 images
Already fetched 6304 images
Already fetched 6305 images
Already fetched 6306 images
Already fetched 6307 images
Already fetched 6308 images
Already fetched 6309 images
Already fetched 6310 images
Already fetched 6311 images
Already fetched 6312 images
Already fetched 6313 images
Already fetched 6314 images
Already fetched 6315 images
Already fetched 6316 images
Already fetched 6317 images
Already fetched 6318 images
Already fetched 6319 images
Already fetched 6320 images
Already fetched 6321 images
Already fetched 6322 images
Already fetched 6323 images
Already fetched 6324 images
Already fetched 6325 images
Already fetched 6326 images
Already fetched 6327 images
Already fetched 6328 images
Already fetched 6329 images
Already fetched 6330

Setup path to data directory and cache directory path to store preprocessed data.

In [4]:
DATA_DIR = 'neurovault/collection_1952/'
CACHE_DIR = 'neurovault/preprocessed_collection_1952/'

Setup dataset parameters.

In [5]:
DOWNSAMPLE_SCALE = 0.25
BATCH_SIZE = 128

Load the dataset into memory.

In [7]:
brainpedia = BrainPedia(data_dirs=[DATA_DIR],
                        cache_dir=CACHE_DIR,
                        scale=DOWNSAMPLE_SCALE)
all_brain_data, all_brain_data_tags = brainpedia.all_data()
brainpedia_batch_generator = BrainPedia.batch_generator(all_brain_data, all_brain_data_tags, BATCH_SIZE, CUDA)
brain_data_shape, brain_data_tag_shape = brainpedia.sample_shapes()

### Plot Example Data Point

Grab an example brain volume and upsample.

In [8]:
all_brain_data.shape

(6573, 1, 13, 15, 11)

In [9]:
example_brain_data = all_brain_data[0].squeeze()
example_brain_data.shape

(13, 15, 11)

In [10]:
upsampled_example_brain_data = invert_preprocessor_scaling(example_brain_data, brainpedia.preprocessor)
upsampled_example_brain_data.shape

(53, 63, 46, 1)

Grab the task description tags for the example brain volume and decode the one-hot encoded tags vector to their english descriptions.

In [11]:
all_brain_data_tags.shape

(6573, 19)

In [12]:
example_brain_data_tags = all_brain_data_tags[0]
example_brain_data_tags

array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [13]:
example_brain_data_decoded_tags = brainpedia.preprocessor.decode_label(example_brain_data_tags)
example_brain_data_decoded_tags

['visual words', 'left hand', 'visual']

Plot example brain volume and their task description tags.

In [ ]:
figure = plt.figure(figsize=(8, 3))
plotting.plot_glass_brain(stat_map_img=upsampled_example_brain_data,
                          threshold='auto',
                          title="Example Brain Volume with Tags: {0}".format(example_brain_data_decoded_tags),
                          figure=figure)
plt.show()

# Models

Setup model hyperparameters.

In [15]:
MODEL_DIMENSIONALITY = 64
CONDITONING_DIMENSIONALITY = 5

CRITIC_UPDATES_PER_GENERATOR_UPDATE = 1

NOISE_SAMPLE_LENGTH = 128

Construct models.

NOTE: Depending on the hyperparameter settings above, the dimensionality of the label vector, noise vector, and layers may not exactly match the figure below.

![ICW-fMRI-GAN Model Architecture](models/resources/architecture.png)

In [16]:
generator = Generator(input_size=NOISE_SAMPLE_LENGTH,
                      output_shape=brain_data_shape,
                      dimensionality=MODEL_DIMENSIONALITY,
                      num_classes=brain_data_tag_shape[0],
                      conditioning_dimensionality=CONDITONING_DIMENSIONALITY,
                      cudaEnabled=CUDA)
critic = Critic(dimensionality=MODEL_DIMENSIONALITY,
                num_classes=brain_data_tag_shape[0],
                conditioning_dimensionality=CONDITONING_DIMENSIONALITY,
                cudaEnabled=CUDA)

#### Load and Save Pretrained Models

In [ ]:
generator_weight_path = 'models/weights/generator.pt'
critic_weight_path = 'models/weights/critic.pt'
epoch_weight_path = 'models/weights/epoch.npy'

In [ ]:
def save_checkpoint(epoch):
    torch.save(generator.state_dict(), generator_weight_path)
    torch.save(critic.state_dict(), critic_weight_path)
    np.save(epoch_weight_path, epoch)

In [ ]:
def load_checkpoint():
    epochs_trained = 0
    if os.path.exists(generator_weight_path) and os.path.exists(critic_weight_path) and os.path.exists(epoch_weight_path):
        generator.load_state_dict(torch.load(generator_weight_path))
        critic.load_state_dict(torch.load(critic_weight_path))
        epochs_trained = np.load(epoch_weight_path) + 1
        print("Loaded models with {} epochs pretrained...".format(epochs_trained))
    else:
        print("Starting from scratch...")
    return epochs_trained

# Training

In [ ]:
NUM_EPOCHS_TO_TRAIN = 5
LAMBDA = 10

Objective to optimize:

![ICW-fMRI-GAN Model Objective](models/resources/objective.png)

In [ ]:
def noise(size, cuda=False):
    noise = torch.from_numpy(np.random.normal(0.0, size=size)).float()
    if cuda:
        noise = noise.cuda()
    return noise

In [ ]:
def train(epoch):
    
    sum_critic_loss = 0.0
    sum_generator_loss = 0.0
    num_batches_in_epoch = all_brain_data.shape[0] // BATCH_SIZE
    
    for _ in tqdm(range(num_batches_in_epoch)):
        
        data_batch, label_batch = next(brainpedia_batch_generator)
        data_batch = Variable(data_batch)
        label_batch = Variable(label_batch)
                    
        # (1) Train critic to distinguish between real and synthetic brain samples
        for critic_step in range(CRITIC_UPDATES_PER_GENERATOR_UPDATE):

            # Generate synthetic data batch for true labels
            critic_noise_batch = Variable(noise(size=(label_batch.shape[0], NOISE_SAMPLE_LENGTH), cuda=CUDA))
            synthetic_data_batch = generator(critic_noise_batch, label_batch)
            
            # Train critic to distinguish between the real and synthetic brain samples
            critic_loss = critic.train(data_batch, synthetic_data_batch, label_batch, LAMBDA)
            sum_critic_loss += critic_loss.item()

        # (2) Train generator to fool critic
        generator_noise_batch = Variable(noise(size=(label_batch.shape[0], NOISE_SAMPLE_LENGTH), cuda=CUDA))
        synthetic_data_batch = generator(generator_noise_batch, label_batch)
        critic_predictions = critic(synthetic_data_batch, label_batch)
        generator_loss = generator.train(critic_predictions)
        sum_generator_loss += generator_loss.item()
                        
    return sum_critic_loss / num_batches_in_epoch, sum_generator_loss / num_batches_in_epoch

In [ ]:
def visualize(epoch):
    
    for data_batch, label_batch in brainpedia_batch_generator:
        
        # Visualize real samples vs synthetic samples
        num_samples_to_visualize = min(data_batch.shape[0], 5)
        fig = plt.figure(figsize=(10*num_samples_to_visualize, 10))
        
        # Generate synthetic samples for labels
        noise_g = Variable(noise(size=(label_batch.shape[0], NOISE_SAMPLE_LENGTH), cuda=CUDA))
        synthetic_data_batch = generator(noise_g, label_batch)
        
        # Visualize comparison between real and synthetic samples
        subplot_idx = 1
        
        for sample_idx in range(num_samples_to_visualize):
            
            data = data_batch[sample_idx]
            synthetic_data = synthetic_data_batch[sample_idx]
            label = label_batch[sample_idx]
            
            data = data.cpu().data.numpy().squeeze()
            synthetic_data = synthetic_data.cpu().data.numpy().squeeze()
            label = brainpedia.preprocessor.decode_label(label)
            
            # Upsample 
            upsampled_data = invert_preprocessor_scaling(data, brainpedia.preprocessor)
            upsampled_synthetic_data = invert_preprocessor_scaling(synthetic_data, brainpedia.preprocessor)

            # Visualize samples
            ax = fig.add_subplot(2, num_samples_to_visualize, subplot_idx)
            plotting.plot_glass_brain(stat_map_img=upsampled_data,
                                      threshold='auto',
                                      title="True {0}".format(label),
                                      axes=ax)
            ax = fig.add_subplot(2, num_samples_to_visualize, subplot_idx+num_samples_to_visualize)
            plotting.plot_glass_brain(stat_map_img=upsampled_synthetic_data,
                                      threshold='auto',
                                      title="Synthetic {0}".format(label),
                                      axes=ax)
            subplot_idx += 1
        
        plt.show()
        break

In [ ]:
epochs_pretrained = load_checkpoint()

for epoch in range(epochs_pretrained, epochs_pretrained+NUM_EPOCHS_TO_TRAIN):
    
    # Train for an epoch
    critic_loss, generator_loss = train(epoch)
    
    print("========= EPOCH {0} ==========".format(epoch))
    print("CRITIC LOSS   : {0:.4f}".format(critic_loss))
    print("GENERATOR LOSS: {0:.4f}\n".format(generator_loss))
    
    # Checkpoint model
    save_checkpoint(epoch)
    
    # Visualize model output
    visualize(epoch)
    